In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Plots

using Dates
using HDF5

In [2]:
function DMD_SVD(Y, r, Δt)
    
    X₋ = Y[:,1:end-1]
    X₊ = Y[:,2:end]
    U, Σ, V = svd(X₋)
    
    Uʳ = U[:, 1:r] #12 x 4
    Σʳ = diagm(Σ[1:r])
    Vʳ = V[:, 1:r]
    Ã = Uʳ' * X₊ * Vʳ / Σʳ
    Λ, W = eigen(Ã)
    Φ = X₊ * Vʳ / Σʳ * W
    Ω = log.(Λ)/Δt
    x₁ = X₋[:,1]
    b₁ = Φ \ x₁
    
    return Φ, Ω, b₁, Ã

end   

function DMD_reconstruct(Φ, Ω, b₁, Δt, steps)
    
    Yᵈᵐᵈ = hcat([real.(Φ * (b₁ .* exp.(Ω * (i * Δt)))) for i in 0:steps]...)
    
    return Yᵈᵐᵈ

end

DMD_reconstruct (generic function with 1 method)

In [3]:
function find_smallest_second_index(Y, threshold)
    # Find all indices where abs(x) > 1 + threshold
    indices = findall(x -> abs(x) > 1 + threshold, Y)

    # Extract the second (column) indices
    second_indices = [ind[2] for ind in indices]

    # Return the smallest second index if found, otherwise return nothing
    if !isempty(second_indices)
        return minimum(second_indices)
    else
        return nothing  # No elements found matching the condition
    end
end

find_smallest_second_index (generic function with 1 method)

In [40]:
function DMD_ERA_physicality(γᵢ, n, time_multiplier, threshold)
        data_dir = "../DATA/"
    
        γᵢ = string(γᵢ)
        γᵢᶠ= parse(Float64, γᵢ)
    
        tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)

        ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
        tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ
        
        lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
        lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series
    
        tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
        tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 
    
        bᵉ = LiPoSID.bloch(ρᵉ[1:lᵐᵃˣ])
        bᵍ = LiPoSID.bloch(ρᵍ[1:lᵐᵃˣ])
        bˣ = LiPoSID.bloch(ρˣ[1:lᵐᵃˣ])
        bʸ = LiPoSID.bloch(ρʸ[1:lᵐᵃˣ])
    
        Y = [bᵉ; bᵍ; bˣ; bʸ]
    
        t = convert.(Float64, tᵉᶠ[1:lᵐᵃˣ])
        Δt = t[2]-t[1]

        # DMD (Dynamic mode decomposition)
        
        Φ, Ω, b₁, Aᴰᴹᴰ = DMD_SVD(Y, n, Δt)

        Aᴰᴹᴰc = log(Aᴰᴹᴰ)/Δt
        Λᴰᴹᴰ, Wᴰᴹᴰ = eigen(Aᴰᴹᴰc)

        Yᴰᴹᴰ = DMD_reconstruct(Φ, Ω, b₁, Δt, length(t)*time_multiplier)

        index_DMD = find_smallest_second_index(Yᴰᴹᴰ, threshold)

        # Check if index_ERA is nothing
        if index_DMD !== nothing
            # Perform the calculation if a valid index is found
            inverse_gamma_time_DMD = index_DMD * Δt * γᵢᶠ
            println("inverse_gamma_time_DMD: ", inverse_gamma_time_DMD)
        else
            inverse_gamma_time_DMD = []
            # Handle the case where no valid index was found
            println("No valid index found, inverse_gamma_time_DMD cannot be calculated.")
        end
            
    
        #bᵉᴰᴹᴰ = Yᴰᴹᴰ[1:3,:]
        #bᵍᴰᴹᴰ = Yᴰᴹᴰ[4:6,:]
        #bˣᴰᴹᴰ = Yᴰᴹᴰ[7:9,:]
        #bʸᴰᴹᴰ = Yᴰᴹᴰ[10:12,:]
    
        #ρᵉᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bᵉᴰᴹᴰ)
        #ρᵍᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bᵍᴰᴹᴰ)
        #ρˣᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bˣᴰᴹᴰ)
        #ρʸᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bʸᴰᴹᴰ)
    
        #ρᴰᴹᴰ = [ρᵉᴰᴹᴰ, ρᵍᴰᴹᴰ, ρˣᴰᴹᴰ, ρʸᴰᴹᴰ]
    
        # ERA (Eigenvalue Realization Algorithm)
    
        Aᴱᴿᴬ, Cᴱᴿᴬ, x₀ᴱᴿᴬ, Σᴱᴿᴬ = LiPoSID.lsid_n_ACx0Σ(Y, Δt, n) 

        Aᴱᴿᴬc = log(Aᴱᴿᴬ)/Δt

        Λᴱᴿᴬ, Wᴱᴿᴬ = eigen(Aᴱᴿᴬc)
    
        Yᴱᴿᴬ = LiPoSID.propagate_LTI(Aᴱᴿᴬ, Cᴱᴿᴬ, x₀ᴱᴿᴬ, n, length(t)*time_multiplier)

        index_ERA = find_smallest_second_index(Yᴱᴿᴬ, threshold)

        # Check if index_ERA is nothing
        if index_ERA !== nothing
            # Perform the calculation if a valid index is found
            inverse_gamma_time_ERA = index_ERA * Δt * γᵢᶠ
            println("inverse_gamma_time_ERA: ", inverse_gamma_time_ERA)
        else
            inverse_gamma_time_ERA = []
            # Handle the case where no valid index was found
            println("No valid index found, inverse_gamma_time_ERA cannot be calculated.")
        end
    
        #bᵉᴱᴿᴬ = Yᴱᴿᴬ[1:3,:]
        #bᵍᴱᴿᴬ = Yᴱᴿᴬ[4:6,:]
        #bˣᴱᴿᴬ = Yᴱᴿᴬ[7:9,:]
        #bʸᴱᴿᴬ = Yᴱᴿᴬ[10:12,:]
    
        #ρᵉᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bᵉᴱᴿᴬ)
        #ρᵍᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bᵍᴱᴿᴬ)
        #ρˣᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bˣᴱᴿᴬ)
        #ρʸᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bʸᴱᴿᴬ)

        #ρᴱᴿᴬ = [ρᵉᴱᴿᴬ, ρᵍᴱᴿᴬ, ρˣᴱᴿᴬ, ρʸᴱᴿᴬ]
        
        return inverse_gamma_time_DMD, inverse_gamma_time_ERA
        
    end

DMD_ERA_physicality (generic function with 1 method)

In [51]:
evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]
γᵢ = γ[1]

"0.079477"

In [52]:
inverse_gamma_time_DMD, inverse_gamma_time_ERA = DMD_ERA_physicality(γᵢ, 4, 100, 1e-6)

No valid index found, inverse_gamma_time_DMD cannot be calculated.
inverse_gamma_time_ERA: 15.884273220000004


(Any[], 15.884273220000004)

In [56]:
println("Times in 1/γ when model predictions bacame unphysical:")

DMD_unphysical_times = []
ERA_unphysical_times = []

for γᵢ in γ
    println("γ=",γᵢ)
    inverse_gamma_time_DMD, inverse_gamma_time_ERA = DMD_ERA_physicality(γᵢ, 5, 100, 1e-6)
    push!(DMD_unphysical_times, inverse_gamma_time_DMD)
    push!(ERA_unphysical_times, inverse_gamma_time_ERA)
    println("DMD:", inverse_gamma_time_DMD)
    println("ERA:", inverse_gamma_time_ERA)
end

Times in 1/γ when model predictions bacame unphysical:
γ=0.079477
No valid index found, inverse_gamma_time_DMD cannot be calculated.
inverse_gamma_time_ERA: 9.624664700000002
DMD:Any[]
ERA:9.624664700000002
γ=0.25133
No valid index found, inverse_gamma_time_DMD cannot be calculated.
No valid index found, inverse_gamma_time_ERA cannot be calculated.
DMD:Any[]
ERA:Any[]
γ=0.79477
No valid index found, inverse_gamma_time_DMD cannot be calculated.
inverse_gamma_time_ERA: 2.282976825000002
DMD:Any[]
ERA:2.282976825000002
γ=2.5133
No valid index found, inverse_gamma_time_DMD cannot be calculated.
No valid index found, inverse_gamma_time_ERA cannot be calculated.
DMD:Any[]
ERA:Any[]
γ=7.9477
No valid index found, inverse_gamma_time_DMD cannot be calculated.
inverse_gamma_time_ERA: 23.5251919999997
DMD:Any[]
ERA:23.5251919999997
γ=25.133
inverse_gamma_time_DMD: 0.00125665
inverse_gamma_time_ERA: 0.00125665
DMD:0.00125665
ERA:0.00125665
γ=79.477
inverse_gamma_time_DMD: 0.00397385
inverse_gamma_

In [57]:
DMD_unphysical_times

8-element Vector{Any}:
  Any[]
  Any[]
  Any[]
  Any[]
  Any[]
 0.00125665
 0.00397385
 0.012566500000000001

In [58]:
ERA_unphysical_times

8-element Vector{Any}:
  9.624664700000002
   Any[]
  2.282976825000002
   Any[]
 23.5251919999997
  0.00125665
  0.00397385
  0.012566500000000001